In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import seaborn as sns
from datetime import datetime, time
from collections import namedtuple

In [2]:
def accept_float(value):
    if value is None:
        return np.NaN
    try:
        return float(value)
    except ValueError:
        return np.NaN
    
def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False
    
def reject_float(value):
    if value is None:
        return np.NaN
    if is_float(value):
        return np.NaN
    else:
        return value
    
def get_month_year(timestamp):
    try:
        # 2020-04-24 11:50:39 UTC
        ft = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S %Z')
        return datetime.strftime(ft, "%Y-%m")
    except:
        print('error')
        return None


In [3]:
sales = pd.read_csv('data/electronics_sales.csv')
sales_copy = sales.copy()

display(sales.dtypes)
sales.head()

event_time        object
order_id           int64
product_id         int64
category_id      float64
category_code     object
brand             object
price            float64
user_id          float64
dtype: object

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
1,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,NaN,karcher,217.57,1.515916e+18


In [4]:
sales.head()

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
1,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,NaN,karcher,217.57,1.515916e+18


In [5]:
sales.describe()

,order_id,product_id,category_id,price,user_id
count,2.633521e+06,2.633521e+06,2.201567e+06,2.201567e+06,5.641690e+05
mean,2.361783e+18,1.674080e+18,2.273827e+18,1.540932e+02,1.515916e+18
std,1.716538e+16,3.102249e+17,2.353247e+16,2.419421e+02,2.379057e+07
min,2.294360e+18,1.515966e+18,2.268105e+18,0.000000e+00,1.515916e+18
25%,2.348807e+18,1.515966e+18,2.268105e+18,1.456000e+01,1.515916e+18
50%,2.353254e+18,1.515966e+18,2.268105e+18,5.553000e+01,1.515916e+18
75%,2.383131e+18,1.515966e+18,2.268105e+18,1.967400e+02,1.515916e+18
max,2.388441e+18,2.388434e+18,2.374499e+18,5.092590e+04,1.515916e+18


In [6]:
display(sales.isna().sum()/sales.shape[0])


event_time       0.000000
order_id         0.000000
product_id       0.000000
category_id      0.164021
category_code    0.232465
brand            0.192140
price            0.164021
user_id          0.785774
dtype: float64

In [7]:
sales.loc[sales.category_id.isna(),:]

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
1812,2020-05-02 11:12:20 UTC,2300138854784434309,1515966223509353669,NaN,0.02,1515915625453675584,NaN,NaN
3633,2020-05-06 07:37:55 UTC,2302930038149874117,1515966223509128764,NaN,38.17,1515915625455624044,NaN,NaN
3918,2020-05-06 22:36:13 UTC,2303382163636093788,1515966223510204743,NaN,0.02,1515915625441101795,NaN,NaN
6015,2020-05-10 09:41:54 UTC,2305891548316827762,1515966223509353669,NaN,0.02,1515915625441980762,NaN,NaN
7802,2020-05-13 03:09:39 UTC,2307868447599493180,1515966223510600539,NaN,0.02,1515915625441293616,NaN,NaN
...,...,...,...,...,...,...,...,...
2632423,2020-11-19 11:34:56 UTC,2388440981134693047,2388434452476082043,NaN,16.18,1515915625514803713,NaN,NaN
2632434,2020-11-19 05:38:31 UTC,2388440981134693055,1515966223523303300,NaN,13.87,1515915625514803718,NaN,NaN
2632437,2020-11-19 11:21:22 UTC,2388440981134693056,1515966223523303302,NaN,57.87,1515915625514803719,NaN,NaN
2632472,2020-11-19 09:54:22 UTC,2388440981134693073,2388434452476685318,NaN,5.76,1515915625514155115,NaN,NaN


In [8]:
sales.price.fillna(sales.category_code, inplace=True)
sales.user_id.fillna(sales.brand, inplace=True)

sales.user_id = sales.user_id.apply(accept_float)
# print(sales[sales.user_id=='pastel'])

sales.user_id.fillna(0, inplace=True)
sales.price = sales.price.apply(accept_float)
sales.price.fillna(0, inplace=True)
sales.category_id.fillna(0, inplace=True)

sales.brand = sales.brand.apply(reject_float)
sales.brand.fillna('unknown', inplace=True)
sales.brand.loc[sales.brand=='none'] = 'unknown'
display(sales.brand.unique())


/var/folders/h3/nfz97p4s4nbfn9k6j6f6cknh0000gn/T/ipykernel_22026/2393954873.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales.brand.loc[sales.brand=='none'] = 'unknown'


array(['samsung', 'huawei', 'karcher', 'maestro', 'apple', 'lg',
       'polaris', 'intel', 'philips', 'asus', 'unknown', 'epson', 'sbs',
       'geyzer', 'tefal', 'kingston', 'ava', 'barjher', 'beurer',
       'schwiizer', 'scarlett', 'caso', 'nokia', 'vivo', 'aoc',
       'logitech', 'maxwell', 'vitek', 'panasonic', 'moulinex', 'sony',
       'bork', 'imetec', 'sjcam', 'braun', 'lenovo', 'redmond', 'xiaomi',
       'globber', 'rondell', 'iqos', 'lavazza', 'razer', 'delux',
       'prestigio', 'ausini', 'rastar', 'steelseries', 'microlab',
       'thomas', 'trust', 'ariston', 'fissman', 'ship', 'hyperx', 'neo',
       'bloody', 'hp', 'tp-link', 'toshiba', 'honor', 'akvafor',
       'jandeks', 'technodom', 'x-game', 'e.gov', 'perilla', 'xbox',
       'usams', 'varta', 'beko', 'jbl', 'muljhtidom', 'genius', 'apollo',
       'kenwood', 'altel', 'thermex', 'gezatone', 'bosch', 'rowenta',
       'inhouse', 'berghoff', 'svetocopy', 'plantronics', 'birjusa',
       'peterhof', 'd-link', 'chi

In [9]:
display(sum(sales.category_code.unique()=='none'))


0

In [10]:
sales.category_code = sales.category_code.apply(reject_float)

sales.category_code.fillna('unknown', inplace=True)

display(sales.isna().sum()/sales.shape[0])

event_time       0.0
order_id         0.0
product_id       0.0
category_id      0.0
category_code    0.0
brand            0.0
price            0.0
user_id          0.0
dtype: float64

In [11]:
sales.category_code.unique()

array(['electronics.tablet', 'electronics.audio.headphone', 'unknown',
       'furniture.kitchen.table', 'electronics.smartphone',
       'appliances.kitchen.refrigerators', 'appliances.personal.scales',
       'electronics.video.tv', 'computers.components.cpu',
       'computers.notebook', 'computers.peripherals.monitor',
       'computers.peripherals.printer', 'appliances.kitchen.kettle',
       'computers.components.memory', 'electronics.clocks',
       'country_yard.weather_station', 'appliances.kitchen.blender',
       'accessories.bag', 'computers.peripherals.keyboard',
       'computers.peripherals.mouse', 'appliances.iron',
       'appliances.kitchen.mixer', 'electronics.telephone',
       'appliances.kitchen.meat_grinder',
       'appliances.personal.hair_cutter',
       'appliances.environment.air_heater', 'kids.skates',
       'appliances.kitchen.washer', 'appliances.kitchen.microwave',
       'furniture.bedroom.pillow', 'appliances.environment.vacuum',
       'appliances.en

In [17]:
sum(sales.brand.unique() == 'other')

0

In [13]:
sales = sales.astype({
    'event_time':'string',
    'category_id':'int64', 
    'category_code':'string',
    'brand':'string', 
    'user_id':'int64'
})

In [14]:
sales['year_month'] = sales['event_time'].apply(get_month_year)
# sales.to_parquet('total_sales.parquet')
# sales.head()

In [15]:
months = sales.year_month.unique()
for m in months:
    sales_m = sales[sales.year_month == m]
    sales_m.to_parquet(f'sales_{m}.parquet')

In [16]:
sales.year_month.unique()

array(['2020-04', '2020-05', '2020-06', '1970-01', '2020-07', '2020-01',
       '2020-02', '2020-03', '2020-08', '2020-09', '2020-10', '2020-11'],
      dtype=object)